In [13]:
#!conda activate py38_estnltk1.7
#hpc serveris peaks sobima estnltk_collocations_py38
#!conda install --channel conda-forge pygraphviz
#!conda install -c conda-forge ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

# Verb - pattern kogumine koondkorpusest


# v30
Kollokatsiooniandmete salvestamisel tabelisse jäeti välja kollokatsioonid, milles:

* verb oli umbisikuline (<code>feats</code> sisaldas parameetrit <code>imps</code>);
* verbi aeg polnud ükski järgnevatest: <code>past</code>, <code>impf</code>, <code>pres</code> (<code>feats</code> ei sisaldanud parameetreid <code>past</code> ega <code>impf</code> ega  <code>pres</code>).

### Tabelid

### TABEL1 transaction_head

| väli | tüüp  |  kirjeldus | näide | märkus |
| --- | --- | --- | --- | --- |
| id | int | rea <br/>unikaalne ID| *56* | |
| sentence_id | int | lause id andmebaasis| | |
| verb_id | int | verbi asukoht lauses | | |
| verb | text |verbi lemma | | |
| verb_compound | text | verbi määrsõnad| alla,peale| eraldajaks koma |
| obl_root_id | int | obl-fraasi juure asukoht lauses| | |


### TABEL2 transaction

| väli | tüüp  |  kirjeldus | näide | märkus |
| --- | --- | --- | --- | --- |
| id | int | rea <br/>unikaalne ID| *56* | |
| head_id | int| rea transaction_head.id  | | |
| loc | int | sõna asukoht lauses | | |
| deprel | text | sõna deprel | | |
| form | text | sõna vorm  | | |
| lemma |  text | sõna lemma | | |
| pos | text | sõna sõnaliik | | |
| feats | text|  sõna morf kategooriad alfabeetilises järjekorras |  add,sg| |






In [14]:
%load_ext autoreload
%autoreload 2

from estnltk.storage.postgres import LayerQuery

from datetime import datetime
# functions for creating database and collecting collocations
from collect_functions import *

from data_helpers.db_reader import DbReader

#collection_name = 'koondkorpus_sentences'
#BATCH_SIZE = 50000

collection_name = 'koondkorpus_sentences_test_5000_sg_thread'
BATCH_SIZE = 10000

TYPE = 'verb_pattern_obl'
TABLENAME = f'{TYPE}'

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"v21_{collection_name}_{TYPE}_collocations_{date_time}.db"
my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


my_db_reader = DbReader(pgpass_file='~/.pgpass',\
                          schema='estonian_text_corpora',\
                          role='estonian_text_corpora_read',\
                          temporary=False,\
                          collection_name=collection_name)
my_db_reader.set_layers(['v172_stanza_syntax', 'v172_obl_phrases', 'v172_clauses'])

# vaatame ainult neid lauseid, kus on mõni tegusõna ka
# NB! kas saab olla obl, kus ülemus ei ole tegusõna
additional_filters = [LayerQuery('morph_analysis', partofspeech='V')]
# sisaldab OBL fraasi
additional_filters.append(LayerQuery('v172_stanza_syntax', deprel='obl'))


data = []
count = 0
for collection_id, text in my_db_reader.get_collections(shuffle=False, queries=additional_filters, progressbar='ascii'):
    count += 1
    collocations, = extract_something(text, collection_id, data )
    
    if not count == 0 and not count % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(data, collection_id)
        data = []


# saving last batch
my_sqlite_db.save_coll_to_db(data, collection_id)
my_sqlite_db.index_fields()

print('done.')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO:storage.py:57: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:108: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


collection_id: 14294:  11%|#1        | 10044/90854 [00:40<05:25, 248.51doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 14226


collection_id: 29042:  22%|##2       | 20031/90854 [01:24<04:58, 237.64doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 29004


...

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 126889


collection_id: 153370:  99%|#########9| 90031/90854 [09:37<00:05, 156.02doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 153164


collection_id: 159735: 100%|##########| 90854/90854 [09:39<00:00, 156.72doc/s]


andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159735
